## Data Preprocessing
Prerequisites:
- Python experience, including a basing understanding of python syntax, loops, conditional statements, functions, and data types in python
- Some background in statistics is helpful

Goals for this session:
- Learn to perform basic exploratory data analysis (EDA) and data visualization
- Identify outliers, handle missing values, and perform other common data operations such as normalization, interpolation, and filtering
- Understand the intuition behind various preprocessing techniques for both categorical and continuous features